# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re
import scipy.stats as sts
import collections as c

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)


,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
df['Результат анализа'][2] = 20

*Практически бесцветной можно считать лишь такую воду, цветность которой не воспринимается глазом и не превышает 20 градусов по нормативам СанПиНа.*

In [4]:
# Ваше решение
df['Норм'] = df['Норматив'].apply(lambda x: ' '.join(x.split(' ')[0:2]))
df['Знач'] = df['Норматив'].apply(lambda x: ''.join(x.split(' ')[2]))

In [5]:
df['Знач'][[1]]=df['Знач'][[1]].apply(lambda x: x.split('-')[1])
df['Знач'][[0,3]]=df['Знач'][[0,3]].apply(lambda x: x.split('-'))

In [6]:
df.loc[df['Знач'].apply(lambda x: isinstance(x, str)),'Знач'] = df.loc[df['Знач'].apply(lambda x: isinstance(x, str)),'Знач'].apply(lambda x: '.'.join(x.split(',')))

In [7]:
df['Результат анализа']=df['Результат анализа'].astype('float64')
df.loc[df['Знач'].apply(lambda x: isinstance(x, str)),'Знач']=df.loc[df['Знач'].apply(lambda x: isinstance(x, str)),'Знач'].astype('float64')

In [34]:
df

,Показатель,Единица измерений,Результат анализа,Норматив,Норм,Знач,Итог
0,pH,единицы pH,8.400,в пределах 6-9,в пределах,"[6, 9]",Результат анализа в норме
1,Запах,баллы,1.000,не более 2-3,не более,3.0,Результат анализа в норме
2,Цветность,градусы,20.000,не более 30,не более,30.0,Результат анализа в норме
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,"[7, 10]",Результат анализа в норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",не более,1.5,Результат анализа в норме
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,3.3,Результат анализа в норме
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45.0,Результат анализа в норме
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",не более,3.5,Результат анализа в норме
8,Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350.0,Результат анализа в норме
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,не более,500.0,Результат анализа в норме


In [29]:
df['Итог']=df.apply(lambda x:  'Результат анализа в норме' if x['Норм'] == 'не более' and x['Результат анализа'] <= x['Знач'] else ('None' if x['Норм'] == 'в пределах' else ('Результат неудовлетворителен')), axis=1)

In [33]:
df['Итог']=df.apply(lambda x: 'Результат анализа в норме' if x['Норм'] == 'в пределах' and x['Результат анализа'] >= float(df.loc[df['Знач'].apply(lambda x: isinstance(x, list)),'Знач'][0][0])
                    and x['Результат анализа'] <= float(df.loc[df['Знач'].apply(lambda x: isinstance(x, list)),'Знач'][0][1]) else x['Итог'] , axis=1)
df['Итог']=df.apply(lambda x: 'Результат анализа в норме' if x['Норм'] == 'в пределах' and x['Результат анализа'] >= float(df.loc[df['Знач'].apply(lambda x: isinstance(x, list)),'Знач'][3][0])
                    and x['Результат анализа'] <= float(df.loc[df['Знач'].apply(lambda x: isinstance(x, list)),'Знач'][3][1]) else x['Итог'] , axis=1)


In [36]:
df=df.set_index('Показатель').drop(columns={'Норм','Знач'})


In [37]:
df

,Единица измерений,Результат анализа,Норматив,Итог
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,Результат анализа в норме
Запах,баллы,1.000,не более 2-3,Результат анализа в норме
Цветность,градусы,20.000,не более 30,Результат анализа в норме
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Результат анализа в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",Результат анализа в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Результат анализа в норме
Нитраты (по NO3),мг/дм3,24.000,не более 45,Результат анализа в норме
Фосфаты (P),мг/дм3,0.360,"не более 3,5",Результат анализа в норме
Хлориды (Cl),мг/дм3,200.000,не более 350,Результат анализа в норме


### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [162]:
n=1000000
bernoulli_rv1 = sts.bernoulli(0.5555555555555556)
bernoulli_rv2 = sts.bernoulli(0.5)
bernoulli_rv3 = sts.bernoulli(0.42)
z1= bernoulli_rv1.rvs(n)
z2= bernoulli_rv2.rvs(n)
z3= bernoulli_rv3.rvs(n)
p=(pd.Series(zip(z1,z2,z3)))

a=[]

[a.append(1) for x in p[:] if x == (1, 1, 1)]
    
len(a)/1000000

0.117001

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [161]:
n=1000000
bernoulli_rv1 = sts.bernoulli(0.3)
bernoulli_rv2 = sts.bernoulli(0.7)

z1= bernoulli_rv1.rvs(n)
z2= bernoulli_rv2.rvs(n)

p=(pd.Series(zip(z1,z2)))

a=[]

[a.append(1) for x in p[:] if x == (1, 1)]
    
len(a)/1000000

0.210361